# Set

In [105]:
import json
import re
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict
import plotly.io as pio
import os

# === 读取json ===
with open("set_run_summary.json", "r") as f:
    data = json.load(f)

# === 正则解析实验名中的参数 ===
def parse_exp_name(name):
    m_match = re.search(r"_m\((.*?)\)", name)
    a_match = re.search(r"_a\((.*?)\)", name)
    sz_match = re.search(r"_sz\((.*?)\)", name)
    return {
        "m": m_match.group(1) if m_match else None,
        "alpha_init": float(a_match.group(1)) if a_match else None,
        "size": sz_match.group(1) if sz_match else None,
    }



In [ ]:
import json
import re
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict
import plotly.io as pio
import os

# === 读取json ===
with open("set_run_summary.json", "r") as f:
    data = json.load(f)

# === 正则解析实验名中的参数 ===
def parse_exp_name(name):
    m_match = re.search(r"_m\((.*?)\)", name)
    a_match = re.search(r"_a\((.*?)\)", name)
    sz_match = re.search(r"_sz\((.*?)\)", name)
    return {
        "m": m_match.group(1) if m_match else None,
        "alpha_init": float(a_match.group(1)) if a_match else None,
        "size": sz_match.group(1) if sz_match else None,
    }


y_key="forward_policy_entropy_eval"
y_name="Average Forward Policy Entropy"
# y_key="spearman_corr_test"
# y_name="Average Spearman Correlation"
if not os.path.exists(y_key):
    os.makedirs(y_key)

# === 聚合数据 ===
for method,fl in [("tb_gfn", False), ("db_gfn", False), ("db_gfn", True)]:
    groups = defaultdict(list)

    for exp_name, record in data.items():
        if record["method"] == method and record["size"] == 'large' and record["fl"] == fl:
            alpha = record["alpha_init"]
            if 0.4 <= alpha <= 0.9:
                steps = record["step"]
                values = record[y_key]
                groups[alpha].append(values)

    # === 计算均值 ===
    mean_results = {}
    for alpha, runs in groups.items():
        runs = np.array(runs)  # shape: (n_runs, n_steps)
        mean_results[alpha] = runs.mean(axis=0)

    # steps = record["step"]  # 假设所有实验的 step 相同
    # 1) 计算均值之后 & 使用前：把 steps 与曲线都截到 9k
    steps = np.asarray(record["step"])
    LIM = 9000
    mask = steps <= LIM

    for a in list(mean_results.keys()):
        mean_results[a] = np.asarray(mean_results[a])[mask]

    steps = steps[mask]  # 以后都用截断后的 steps

    # === Plotly 绘制折线图 ===
    fig = go.Figure()

    for alpha in sorted(mean_results.keys()):
        fig.add_trace(go.Scatter(
            x=steps,
            y=mean_results[alpha],
            mode="lines",
            name=f"α={alpha}"
        ))

    # —— 生成曲线略 —— #
    # 标题：用 Unicode 而不是 MathJax，保证能控制字号
    def method_title(method, fl):
        if method == "tb_gfn":
            return "TB"
        elif method == "db_gfn" and not fl:
            return "DB"
        else:
            return "FL-DB"

    title_text = method_title(method, fl)

    # 紧凑但不挤压的范围
    y_min = min(float(np.min(v)) for v in mean_results.values())
    y_max = max(float(np.max(v)) for v in mean_results.values())
    pad = 0.02 * (y_max - y_min if y_max > y_min else 1.0)
    y_range = [y_min - pad, y_max + pad]
    x_min, x_max = (min(steps), max(steps))

    fig.update_layout(
        width=800, height=600,
        # title=None,
        title=dict( text=title_text, font=dict(size=28, color="black"), x=0.5, xanchor="center", y=0.99, yanchor="top"), # 更靠近绘图区 pad=dict(t=0, b=0, l=0, r=0) # 去掉额外留白（关键
        margin=dict(l=80, r=22, t=30, b=70),
        legend=dict(
            x=0.99, y=0.01, xanchor="right", yanchor="bottom",
            orientation="v",
            font=dict(size=28),              # 图例字体
            bgcolor="rgba(255,255,255,0.6)"  # 可读性更好；想更紧凑可去掉
        ),
        template="plotly_white",
        paper_bgcolor="white",
        plot_bgcolor="white",
    )

    # X 轴：强制每 1000 一个刻度，并显示 1k/2k 格式
    fig.update_xaxes(
        range=[x_min, x_max],
        title=dict(text="Step", font=dict(size=28, color="black"), standoff=12),
        tickfont=dict(size=18, color="black"),
        ticks="outside",
        tickmode="linear",
        tick0=0,
        dtick=1000,
        tickformat="~s",
        automargin=True,
    )

    # Y 轴
    fig.update_yaxes(
        range=y_range,
        title=dict(text=y_name, font=dict(size=28, color="black"), standoff=12),
        tickfont=dict(size=18, color="black"),
        ticks="outside",
        automargin=True,
    )

    pio.write_image(fig, f'{y_key}/{method}-fl_{int(fl)}.pdf',
                    format='pdf', width=800, height=600, scale=2)



# mols

In [ ]:
# y_key="forward_policy_entropy_eval"
# y_name="Average Forward Policy Entropy"
y_key="spearman_corr_test"
y_name="Average Spearman Correlation"
y_dir = f'{y_key}_compare'
if not os.path.exists(y_dir):
    os.makedirs(y_dir)

# === 读取 json ===
with open("mols_run_summary.json", "r") as f:
    data = json.load(f)

# === 聚合数据 ===
for method,fl in [("db", False)]:
    groups = defaultdict(list)
    for exp_name, record in data.items():
        if record["objective"] == method and record["fl"] == fl:
            alpha = record["alpha_init"]
            if alpha in [0.5,0.9]:
                steps = record["step"]
                values = record[y_key]
                groups[alpha].append(values)
    for alpha, runs in groups.items():
        print(alpha)
        for run in runs:
            print(run)
    # === 计算均值 ===
    mean_results = {}
    for alpha, runs in groups.items():
        runs = np.array(runs)  # shape: (n_runs, n_steps)
        mean_results[alpha] = runs.mean(axis=0)

    # steps = record["step"]  # 假设所有实验的 step 相同
    # 1) 计算均值之后 & 使用前：把 steps 与曲线都截到 9k
    steps = np.asarray(record["step"])
    LIM = 50000
    mask = steps <= LIM

    for a in list(mean_results.keys()):
        mean_results[a] = np.asarray(mean_results[a])[mask]

    steps = steps[mask]  # 以后都用截断后的 steps

    # === Plotly 绘制折线图 ===
    fig = go.Figure()

    for alpha in sorted(mean_results.keys()):
        fig.add_trace(go.Scatter(
            x=steps,
            y=mean_results[alpha],
            mode="lines",
            name=f"α={alpha}"
        ))

    # —— 生成曲线略 —— #
    # 标题：用 Unicode 而不是 MathJax，保证能控制字号
    # def method_title(method, fl):
    #     if method == "tb":
    #         return "TB"
    #     elif method == "db_gfn" and not fl:
    #         return "DB"
    #     else:
    #         return "FL-DB"

    # title_text = method_title(method, fl)
    title_text=method.upper()

    # 紧凑但不挤压的范围
    y_min = min(float(np.min(v)) for v in mean_results.values())
    y_max = max(float(np.max(v)) for v in mean_results.values())
    pad = 0.02 * (y_max - y_min if y_max > y_min else 1.0)
    y_range = [y_min - pad, y_max + pad]
    x_min, x_max = (min(steps), max(steps))

    fig.update_layout(
        width=800, height=600,
        # title=None,
        title=dict( text=title_text, font=dict(size=28, color="black"), x=0.5, xanchor="center", y=0.99, yanchor="top"), # 更靠近绘图区 pad=dict(t=0, b=0, l=0, r=0) # 去掉额外留白（关键
        margin=dict(l=80, r=22, t=30, b=70),
        legend=dict(
            x=0.99, y=0.01, xanchor="right", yanchor="bottom",
            orientation="v",
            font=dict(size=28),              # 图例字体
            bgcolor="rgba(255,255,255,0.6)"  # 可读性更好；想更紧凑可去掉
        ),
        template="plotly_white",
        paper_bgcolor="white",
        plot_bgcolor="white",
    )

    # X 轴：强制每 1000 一个刻度，并显示 1k/2k 格式
    fig.update_xaxes(
        range=[x_min, x_max],
        title=dict(text="Step", font=dict(size=28, color="black"), standoff=12),
        tickfont=dict(size=18, color="black"),
        ticks="outside",
        tickmode="linear",
        tick0=0,
        dtick=1000,
        tickformat="~s",
        automargin=True,
    )

    # Y 轴
    fig.update_yaxes(
        range=y_range,
        title=dict(text=y_name, font=dict(size=28, color="black"), standoff=12),
        tickfont=dict(size=18, color="black"),
        ticks="outside",
        automargin=True,
    )

    pio.write_image(fig, f'{y_dir}/{method}-fl_{int(fl)}.pdf',
                    format='pdf', width=800, height=600, scale=2)



0.5
[-0.6133767521137454, None, None, None, 0.30545385370168243, None, None, None, 0.43749892096624593, None, None, None, 0.5405574223525202, None, None, None, 0.40510553655076925, None, None, None, 0.5170976712974449, None, None, None, 0.4757183982542137, None, None, None, 0.45145566801132647, None, None, None, 0.42435867798161053, None, None, None, 0.45703740881299326, None, None, None, 0.3584458851761846, None, None, None, 0.4484237976079585, None, None, None, 0.528748165170933, None, None, None, 0.4434820354318081, None, None, None, 0.5700994004377926, None, None, None, 0.4157748173830068, None, None, None, 0.5082275019903455, None, None, None, 0.5038202168511738, None, None, None, 0.47143199219949344, None, None, None, 0.5343025601379434, None, None, None, 0.49762651784998324, None, None, None, 0.547125366062833, None, None, None, 0.47892712680750965, None, None, None, 0.4859444193699245, None, None, None, 0.5875807189673671, None, None, None, 0.49110566319978677]
[-0.618187972628

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [ ]:
import json
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict
import plotly.io as pio

# === 读取 json ===
with open("mols_run_summary.json", "r") as f:
    data = json.load(f)

# === 聚合数据：对相同 alpha 的实验取平均 ===
groups_length = defaultdict(list)
groups_entropy = defaultdict(list)

for record in data.values():
    if record["objective"] == "subtb" and record["fl"] is True:
        # 规整到一位小数，避免浮点误差
        alpha = round(float(record["alpha_init"]), 1)
        if 0.1 <= alpha <= 0.9:
            avg_length = np.mean(record["all_samples_avg_length_eval"])
            avg_entropy = np.mean(record["forward_policy_entropy_eval"])
            groups_length[alpha].append(avg_length)
            groups_entropy[alpha].append(avg_entropy)

# 计算每个 alpha 的均值
alpha_list = sorted(groups_length.keys())
mean_length_list = [np.mean(groups_length[a]) for a in alpha_list]
mean_entropy_list = [np.mean(groups_entropy[a]) for a in alpha_list]

# === x 轴刻度：强制显示 0.1-0.9 的 9 个数字 ===
x_ticks = [round(0.1 * i, 1) for i in range(1, 10)]
x_ticktext = [f"{v:.1f}" for v in x_ticks]

# === 绘图 ===
fig = go.Figure()

# 左轴：平均长度
fig.add_trace(go.Scatter(
    x=alpha_list,
    y=mean_length_list,
    mode="lines+markers",
    name="Avg Sample Length",
    line=dict(color="blue"),
    yaxis="y",
    cliponaxis=False   # 防止边缘点被裁剪
))

# 右轴：前向策略熵
fig.add_trace(go.Scatter(
    x=alpha_list,
    y=mean_entropy_list,
    mode="lines+markers",
    name="Forward Policy Entropy",
    line=dict(color="red"),
    yaxis="y2",
    cliponaxis=False
))

# === 布局（压缩上下留白）===
fig.update_layout(
    width=450,
    height=500,
    showlegend=False,
    template="plotly_white",
    margin=dict(t=0, b=0, l=28, r=28),  # 上下 0；左右可按需再调小
    yaxis=dict(
        title="Average Sample Length",
        title_font=dict(size=28, color="blue"),
        tickfont=dict(size=16, color="blue"),
        side="left",
        automargin=True
    ),
    yaxis2=dict(
        title="Average Forward Policy Entropy",
        title_font=dict(size=28, color="red"),
        tickfont=dict(size=16, color="red"),
        overlaying="y",
        side="right",
        automargin=True
    )
)

# === x 轴：强制刻度 + 轻微放宽范围，避免两端点被吞 ===
pad = 0.02  # 如仍被裁，可调到 0.03~0.05
xmin = x_ticks[0] - pad   # 0.08
xmax = x_ticks[-1] + pad  # 0.92
fig.update_xaxes(
    title="α",
    title_font=dict(size=20, color="black"),
    tickfont=dict(size=16, color="black"),
    tickmode="array",
    tickvals=x_ticks,
    ticktext=x_ticktext,
    range=[xmin, xmax],
    automargin=True
)

# === 输出 ===
# fig.show()
pio.write_image(fig, "alpha_dual_axis_avg.pdf", width=500, height=500, scale=2)


In [109]:
def calc(num1,num2):
    print(num2/num1-1)

calc(10.60,15.20)
calc(10.60,23.60)
calc(24.60,26.00)
calc(15.40,35.00)

calc(37.60,35.00)

0.4339622641509433
1.226415094339623
0.05691056910569103
1.2727272727272725
-0.06914893617021278


In [110]:
calc(10.00,14.40)
calc(9.00,25.00)
calc(22.20,26.40)
calc(16.00,39.20)
calc(38.40,40.20)

0.43999999999999995
1.7777777777777777
0.18918918918918926
1.4500000000000002
0.04687500000000022


In [104]:
(36.5*4+50)/5

39.2

In [111]:
13/15

0.8666666666666667